In [28]:
#import packages and create date

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import *
import shutil
import difflib
from pulp import *
import openpyxl

#import datetime
today = date.today()
#today = today - datetime.timedelta(days=1)
today = today.strftime("%m.%d.20%y")
today

'03.08.2022'

In [29]:
## DKNG Player Pool

contest = "PlayerPool_"
os.chdir(r"C:\Users\vchang\Downloads")

downloads = r'C:/Users/vchang/Downloads'
contest_data_loc = r'C:/Users/vchang/Documents/Fantasy Bball/DFS/DKNG_Data/PlayerPool'

shutil.move(downloads + "/"+ "DKSalaries.csv",
           contest_data_loc + "/"+ "DKSalaries.csv")

os.chdir(r"C:/Users/vchang/Documents/Fantasy Bball/DFS/DKNG_Data/PlayerPool")
dkng_file = shutil.copy("DKSalaries.csv", "DKSalaries_"+ contest + str(today)+".csv")
dkng_file = pd.read_csv(r"C:/Users/vchang/Documents/Fantasy Bball/DFS/DKNG_Data/PlayerPool/DKSalaries.csv")
dkng_file.to_excel(r"C:/Users/vchang/Documents/Fantasy Bball/DFS/DKNG_Data/PlayerPool/DKSalaries_"+ contest + str(today)+".xlsx", index=False)
os.remove("DKSalaries.csv")
os.remove("DKSalaries_"+ contest + str(today)+".csv")

In [38]:
## ETR Daily

# changes the working directory to your downloads folder
os.chdir(r"C:\Users\vchang\Downloads")

downloads = r'C:/Users/vchang/Downloads'
data_loc = r'C:/Users/vchang/Documents/Fantasy Bball/DFS/ETR_Data/DraftKings'
daily_file = "DK NBA Projections.csv"

shutil.move(downloads + "/"+ daily_file,
           data_loc + "/"+ daily_file)

os.chdir(data_loc)

df = pd.read_csv(daily_file)

df.to_excel("ETR_Daily_"+str(today)+".xlsx", index=False)
os.remove("DK NBA Projections.csv")

In [39]:
## Joins the DKNG and ETR Files to include DKNG Positions 
    ## ETR does not include G/F/UTIL

os.chdir(r"C:\Users\vchang\Documents\Fantasy Bball\DFS\PlayerPool Analysis")
etr = pd.read_excel(r'C:\Users\vchang\Documents\Fantasy Bball\DFS\ETR_Data\DraftKings\ETR_Daily_' + str(today) + ".xlsx")
dkng = pd.read_excel(r'C:\Users\vchang\Documents\Fantasy Bball\DFS\DKNG_Data\PlayerPool\DKSalaries_PlayerPool_' + str(today) + ".xlsx")

## TO DO - check if merge on only DKNG Roster Position Works
df = pd.merge(etr, 
              dkng[['Name','Roster Position', 'Name + ID']], 
              left_on='Player', 
              right_on='Name')

In [40]:
## Unnests Roster Positions into each Row (explode in Python) ##


## Update Salary to INT 
df['DK Salary'] = df['DK Salary'].str.replace(",","").astype(float)

df['Roster Position'] = df['Roster Position'].astype(str)
df['Roster Position'] = df['Roster Position'].str.split('/')

df = df.explode('Roster Position')

df.head()

df.to_csv("DKNG_final_PlayerPool_ETR_"+str(today)+".csv", index=False)
# os.system("DKNG_final_PlayerPool_ETR_"+str(today)+".xlsx")

In [41]:
df.head()

,Player,Team,Opp,Minutes,DK Position,DK Salary,DK Points,DK Value,DK Ceiling,DK Own,DK Slate,Name,Roster Position,Name + ID
0,Cameron Payne,PHX,ORL,33.0,PG,5300.0,41.8,14.6,50.1,64.6,MAIN,Cameron Payne,PG,Cameron Payne (21591011)
0,Cameron Payne,PHX,ORL,33.0,PG,5300.0,41.8,14.6,50.1,64.6,MAIN,Cameron Payne,G,Cameron Payne (21591011)
0,Cameron Payne,PHX,ORL,33.0,PG,5300.0,41.8,14.6,50.1,64.6,MAIN,Cameron Payne,UTIL,Cameron Payne (21591011)
1,CJ McCollum,NOP,MEM,37.0,PG/SG,9100.0,50.3,7.9,57.4,6.2,MAIN,CJ McCollum,PG,CJ McCollum (21590811)
1,CJ McCollum,NOP,MEM,37.0,PG/SG,9100.0,50.3,7.9,57.4,6.2,MAIN,CJ McCollum,SG,CJ McCollum (21590811)


In [42]:
os.getcwd()

'C:\\Users\\vchang\\Documents\\Fantasy Bball\\DFS\\PlayerPool Analysis'

In [43]:
df = pd.read_csv("DKNG_final_PlayerPool_ETR_"+str(today)+".csv", index_col=['Name + ID', 'Roster Position'], skipinitialspace=True)

legal_assignments = df.index # tuples of (name, pos)
name_set = df.index.unique(0) # a convenience

costs = df['DK Salary'].to_dict()
values = df['DK Points'].to_dict()

# set up LP
draft = pulp.LpVariable.dicts('selected', legal_assignments, cat='Binary')

prob = pulp.LpProblem('the draft', LpMaximize)

#obj
prob += pulp.lpSum([draft[n,p]*values[n,p] for (n,p) in legal_assignments])

#salary cap
prob += pulp.lpSum([draft[n,p]*costs[n,p] for (n, p) in legal_assignments]) <= 50000

# positions
prob += pulp.lpSum([draft[n, p] for (n, p) in legal_assignments if p == 'PG']) == 1
prob += pulp.lpSum([draft[n, p] for (n, p) in legal_assignments if p == 'SG']) == 1
prob += pulp.lpSum([draft[n, p] for (n, p) in legal_assignments if p == 'G']) == 1
prob += pulp.lpSum([draft[n, p] for (n, p) in legal_assignments if p == 'SF']) == 1
prob += pulp.lpSum([draft[n, p] for (n, p) in legal_assignments if p == 'PF']) == 1
prob += pulp.lpSum([draft[n, p] for (n, p) in legal_assignments if p == 'F']) == 1
prob += pulp.lpSum([draft[n, p] for (n, p) in legal_assignments if p == 'C']) == 1
prob += pulp.lpSum([draft[n, p] for (n, p) in legal_assignments if p == 'UTIL']) == 1

# use each player at most only once
result = []
for name in name_set:
    prob += pulp.lpSum([draft[n, p] for (n, p) in legal_assignments if n == name]) <= 1

prob.solve()

for idx in draft:
    if draft[idx].varValue > 0:
        print(idx[0], idx[1])
        
        result.append([idx[0], idx[1]])

Cameron Payne (21591011) G
CJ McCollum (21590811) SG
Deandre Ayton (21590906) C
Jonas Valanciunas (21590872) UTIL
Jordan Nwora (21591326) PF
Jae Crowder (21591032) SF
LaMelo Ball (21590829) PG
Herbert Jones (21591060) F


C:\Users\vchang\Anaconda3\lib\site-packages\pulp\pulp.py:1313: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [44]:
new_list = result
df = pd.DataFrame(new_list)
test = df[[1,0]]
test1 = test.transpose()
test2 = test1.rename(columns=test1.iloc[0])
test3 = test2.drop([1])
df2 = test3[['PG', 'SG', 'SF', 'PF', 'C', 'G', 'F', 'UTIL']]
df2.to_csv("dkng_result_" + str(today) + ".csv", index=None)

In [45]:
df2.head()

,PG,SG,SF,PF,C,G,F,UTIL
0,LaMelo Ball (21590829),CJ McCollum (21590811),Jae Crowder (21591032),Jordan Nwora (21591326),Deandre Ayton (21590906),Cameron Payne (21591011),Herbert Jones (21591060),Jonas Valanciunas (21590872)
